# B-Score: Open Script for IEEE Publication 

This is the complementary code script section for the B-Score Paper published in the IEEE Biomedical and Health Informatics Journal. It is an Open Source script available under the GNU General Public License:

    Copyright (C) 2021  Tomas L Bothe

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>

    Author contact information:
    Tomas L Bothe
    tomas-lucca.bothe@charite.de


Link to the Github Repository: https://github.com/ChariteBothe/IEEE-B-Score-Code

# Functionality

This script allows you to analyse your BP estimation dataset and calculate the base performances needed for B-Score calculation.

If you choose to provide your T-RMSE value, this script will directly allow you to calculate your model's B-Score.
Vice versa you a free to calculate the T-RMSE needed on your dataset to reach differernt B-Scores.

Please note that you have to calculate the following steps seperately for the systolic and diastolic versions of your dataset.

# How to use:

It is highly recommended to run the script in the Google Colaboratory environment to ensure ideal functionality. If you choose to run the script in another environment, please make sure all required libraries are installed on your machine. 

To make sure you have all required libraries installed with the correct versions, please check the publication and the code-cell below for detailed information.

Cells containing necessary imports or utility functions are collapsed. Please run them by pressing Shift + ENTER.

Cells containing optional functionality will be highlighted by explanatory texts beforehand.
If you wish to inspect the code double click on collapsed cells to expand them.

In [ ]:
#@title Only for Google Colab Users: Connect to Google Drive

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#@title Import necessary libraries

import numpy as np
import pandas as pd
import time
import math
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing
from sklearn.model_selection import GroupShuffleSplit
from sklearn_pandas import DataFrameMapper

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.metrics import RootMeanSquaredError

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#@title Import utility functions


# Strips Dataframe of all unused columns
def prep_df(df, handles):
  df = df[handles]
  return df


# Definition DL-Basemodel
def get_model(input_shape):
  model = Sequential([
                      Dense(8, activation = 'relu', kernel_regularizer=regularizers.l2(0.0001), input_shape = input_shape),
                      BatchNormalization(),
                      Dropout(0.5),
                      Dense(6, activation = 'relu', kernel_regularizer=regularizers.l2(0.0001)), 
                      BatchNormalization(),
                      Dropout(0.5),
                      Dense(4, activation = 'relu', kernel_regularizer=regularizers.l2(0.0001)), 
                      BatchNormalization(),
                      Dropout(0.5),
                      Dense(3, activation = 'relu', kernel_regularizer=regularizers.l2(0.0001)),
                      BatchNormalization(),
                      Dense(1)
                      ])
  return model


# Circularize Time and Cal_Time (sin / cos transformation)
def time_circularization(df, Measurement_time, Calibration_time):
  df[Measurement_time] = pd.to_timedelta(df[Measurement_time])
  df[Measurement_time] = df[Measurement_time].dt.total_seconds()
  df[Calibration_time] = pd.to_timedelta(df[Calibration_time])
  df[Calibration_time] = df[Calibration_time].dt.total_seconds()

  df['Time_sin'] = np.sin(2 * np.pi * df[Measurement_time] / 86400)
  df['Time_cos'] = np.cos(2 * np.pi * df[Measurement_time] / 86400)
  df['Cal_Time_sin'] = np.sin(2 * np.pi * df[Calibration_time] / 86400)
  df['Cal_Time_cos'] = np.cos(2 * np.pi * df[Calibration_time] / 86400)
  return df


# Setting patience parameter based on dataset shape
def get_patience(df):
  i = df.shape[0]
  patience = math.ceil(min(200, max(2, (200000 / i))))
  return patience


# Setting k-fold iteration parameter based on ID size
def get_k_fold(df, ID):
  ids = df[ID].unique().shape[0]
  k_fold = math.ceil(min(10, max(3, (ids / 10))))
  return (ids, k_fold)


# Setting shuffle iteration parameter based on dataset shape
def get_shuffles(df):
  i = df.shape[0]
  shuffles = math.ceil(min(4, max(1, (10000 / i))))
  return shuffles


# Setting repeat iteration parameter based on dataset shape
def get_repeats(df):
  i = df.shape[0]
  repeats = math.ceil(min(10, max(2, (50000 / i))))
  return repeats



# Training model and saving the best version
def train_model(model, X_train, y_train, X_val, y_val, patience):
  ES = EarlyStopping(monitor = 'val_RMSE',
                     patience = patience,
                     verbose = 0)
  
  Checkpoint = ModelCheckpoint(filepath = savepath,
                               monitor = 'val_RMSE',
                               verbose = 0,
                               save_freq = 'epoch',
                               save_best_only = True,
                               save_weights_only = True)
  
  LR = 0.1
  for i in range(5):
    if i == 0:
      model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = LR),
                    loss = 'mse',
                    metrics = [RootMeanSquaredError(name = 'RMSE')])      
    else:
      LR = LR * 0.1
      model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = LR),
                    loss = 'mse',
                    metrics = [RootMeanSquaredError(name = 'RMSE')])
      model.load_weights(savepath)
    
    history = model.fit(X_train, y_train, validation_data = (X_val, y_val),
                        batch_size = 256,
                        epochs = 100000,
                        verbose = 0,
                        callbacks = [ES, Checkpoint],
                        shuffle = True)
    

# B-Score calculation
def calculate_B_Scores(B1, B2, M, T):
  B_Score = np.log10(np.sqrt((B1 * M / (T * T)) * (B2 * M / (T * T))))
  return B_Score


# Load dataset
def load_data(filepath, seperator):
  df = pd.read_csv(filepath,
                   sep = seperator)
  df, handles = get_handles(df)
  df = prep_df(df, handles)
  return df, handles


# Get column handles
def get_handles(df):
  handles = []
  handles.append(ID)
  handles.append(Sex)
  handles.append(Age)
  handles.append(Heart_Rate)
  handles.append(Calibration_Heart_Rate)


  if Measurement_time != None and Calibration_time != None:
    df = time_circularization(df, Measurement_time, Calibration_time)
    handles.append('Time_sin')
    handles.append('Time_cos')
    handles.append('Cal_Time_sin')
    handles.append('Cal_Time_cos')

  else:
    handles.append(Measurement_time_sin)
    handles.append(Measurement_time_cos)
    handles.append(Calibration_time_sin)
    handles.append(Calibration_time_cos)

  handles.append(Calibration_BP)
  handles.append(Reference_values)
  return df, handles

# Upload your dataset

Insert the file path to your dataset in the parentheses below. Note that the filepath has to end in a .csv file. Please also select a savepath needed for the evaluation model.

Your dataset should provide a new row for each blood pressure measurement taken and seperate the recorded parameters in columns.

Please indicate which kind of seperator your .csv file is using. The default is ','

In [ ]:
# Load Dataset

filepath = 'Insert your dataset filepath here'
savepath = 'Insert your model savepath here'

seperator = 'Insert your .csv seperator here'

In [ ]:
#@title Inspect your dataset

df = pd.read_csv(filepath,
                 sep = seperator)
df.head(5)

# Define column handles

Please provide the column handles for **your** dataset below **exactly** as they are shown in your dataset above. Write the handles inside the provided paratheses. If you are promted to set the variable to NONE, please do so **without** the parentheses.
The following parameters are expected to be represented as individual columns:

*   Patient ID
*   Patient Sex
*   Patient Age
*   Heart Rate at measurement
*   Calibration Heart Rate
*   Calibration Blood Pressue (systolic or diastolic)
*   Time of Calibration
*   Time of current Measurement
*   Reference Blood Pressure (systolic or diastolic)

It is assumed that you dataset already has calibration times and values. If this is not the case, please revisit the B-Score publication for further information. 

Your measurement and calibration time values will be sine / cosine transformed if they have not been already. Please provide **either** two column handles for uncircularized times or four column handles for circularized times.

Set the reference values variable (reference blood pressure value) to your column depicting **either** SBP or DBP **referece** (e.g. cuff-based) values. Please remark that systolic and diastolic performance can only be evaluated seperately. Please set the BP-type variable to **either** 'SBP' or 'DBP'.

In [ ]:
# Please provide the column handles for your dataset:
ID = 'Insert handle'
Sex = 'Insert handle'
Age = 'Insert handle'
Heart_Rate = 'Insert handle'
Calibration_Heart_Rate = 'Insert handle'
Calibration_BP = 'Insert handle'


# Please provide the column handles for either measurement and calibration
# time values OR four the four circularized time values:
Measurement_time = None
Calibration_time = None
# OR
Measurement_time_sin = 'Insert handle'
Measurement_time_cos = 'Insert handle'
Calibration_time_sin = 'Insert handle'
Calibration_time_cos = 'Insert handle'


# Please provide the ground truth label and the BP-type:
Reference_values = 'Insert handle'
BP_type = 'SBP or DBP'

In [ ]:
#@title Dataset handles and time circularization

handles = []
handles.append(ID)
handles.append(Sex)
handles.append(Age)
handles.append(Heart_Rate)
handles.append(Calibration_Heart_Rate)


if Measurement_time != None and Calibration_time != None:
  df = time_circularization(df, Measurement_time, Calibration_time)
  handles.append('Time_sin')
  handles.append('Time_cos')
  handles.append('Cal_Time_sin')
  handles.append('Cal_Time_cos')

else:
  handles.append(Measurement_time_sin)
  handles.append(Measurement_time_cos)
  handles.append(Calibration_time_sin)
  handles.append(Calibration_time_cos)

handles.append(Calibration_BP)
handles.append(Reference_values)
df = prep_df(df, handles)

In [ ]:
#@title Inspect your dataset

df.head(5)

# K-fold evaluation

After dataset preparation the following cell will load and run the k-fold evaluation process. Depending on your dataset's size three parameters will be determined:

1.   Number of k-fold iterations (based on number of unique individuals in the dataset)
2.   Number of repeats (best of, based on dataset length)
3.   Number of reshuffled iterations (average of,  based on dataset length)


Please note that this may take a while, depending on the chosen dataset size and your machine's computational capacities.

In [ ]:
#@title Determining iteration parameters
k_fold = get_k_fold(df, handles[0])[1]
repeats = get_repeats(df)
shuffles = get_shuffles(df)
patience = get_patience(df)

print('Running for ' + str(k_fold) + ' iterations')
print('Running for ' + str(repeats) + ' repeats')
print('Running for ' + str(shuffles) + ' shuffles')

In [ ]:
#@title Calculation functions

# Function for repeated k-fold iterations
def repeat_k_fold(df, k_fold, repeats, handles, patience):

  start = time.time()
    
  df = prep_df(df, handles)
  test_handle = handles.pop(-1)
  id_handle = handles[0]
  cal_handle = handles[-1]
  train_handle = handles[1 : ]
  train_handle.append('Cohort_Mean')
  del handles

  ids = df[id_handle].unique()
  np.random.shuffle(ids)
  k_ids = np.array_split(ids, k_fold)
  df_list = []
  B_Score_list = []
  RMSE_list = []

  for i in range(k_fold):

    df_new = df[df[id_handle].isin(k_ids[i])]
    df_list.append(df_new)
    del df_new


  B1_RMSE_list = []
  B2_RMSE_list = []
  M_RMSE_list = []
  df_train_shape_list = []
  M_RMSE_single = []


  for i in range(k_fold): 

    df_test = df_list.pop(0)
    df_val = df_list[0]
    df_train = pd.concat(df_list[1 :])
    df_list.append(df_test)
    df_train_shape_list.append(df_train.shape[0])

    Cohort_Mean = df_train[test_handle].mean()
    df_train['Cohort_Mean'] = Cohort_Mean
    df_val['Cohort_Mean'] = Cohort_Mean
    df_test['Cohort_Mean'] = Cohort_Mean

    # Get B1-RMSE and B2-RMSE and append them to list
    B1_RMSE = mean_squared_error(df_test[test_handle], df_test['Cohort_Mean'], squared = False)
    B1_RMSE_list.append(B1_RMSE)
    B2_RMSE = mean_squared_error(df_test[test_handle], df_test[cal_handle], squared = False)
    B2_RMSE_list.append(B2_RMSE)

    # Split into X and y datasets
    X_train = df_train[train_handle]
    y_train = df_train[test_handle]
    X_val = df_val[train_handle]
    y_val = df_val[test_handle]
    X_test = df_test[train_handle]
    y_test = df_test[test_handle]

    # Normalize X on X_train
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    del df_train
    del df_val
    del df_test

    M_list = []
    for _ in range(repeats):
    
      # Get, compile and train model
      model = get_model(X_train[1].shape)
    
    
      train_model(model, X_train, y_train, X_val, y_val, patience)

      # Reload best model
      model.load_weights(savepath)

      # Get M_RMSE and append to list
      M = model.evaluate(X_test, y_test, batch_size = 4096)[1]
      M_list.append(M)
      
      

    M_RMSE_single.append(M_list)
    M_list.sort()
    M_RMSE_list.append(M_list[0])

    end = time.time()
    minutes = round((end - start) / 60, 2)
    print('Time since start of shuffle: ' + str(minutes) + ' minutes')
    print(str(i + 1) + '/' + str(k_fold) + ' of shuffle completed') 


  B1_avg = sum(B1_RMSE_list) / len(B1_RMSE_list)
  B2_avg = sum(B2_RMSE_list) / len(B2_RMSE_list)
  M_avg = sum(M_RMSE_list) / len(M_RMSE_list)

  return B1_avg, B2_avg, M_avg

def run_shuffle(shuffles, repeats, k_fold, filepath, seperator):
  starttime = time.time()
  B1 = []
  B2 = []
  M = []
  print('Running for ' + str(shuffles) + ' shuffles')
  print('Running for ' + str(k_fold) + ' k_fold iterations per shuffle')
  print('Running for ' + str(repeats) + ' repeats per k_fold iteration')

  for k in range(shuffles):
    print('\n\nStart of shuffle ' + str(k + 1) + ' of ' + str(shuffles))
    df, handles = load_data(filepath, seperator)
    B1_avg, B2_avg, M_avg = repeat_k_fold(df, k_fold, repeats, handles, patience)
    B1.append(B1_avg)
    B2.append(B2_avg)
    M.append(M_avg)
    endtime = time.time()
    now = round((endtime - starttime) / 60, 2)
    print('Time since start of total calculation: ' + str(now) + ' minutes')
    print(str(k + 1) + '/' + str(shuffles) + ' of total calculation completed')

  B1 = sum(B1) / len(B1)
  B2 = sum(B2) / len(B2)
  M = sum(M) / len(M)
  return B1, B2, M

In [ ]:
#@title Run calculation

B1_total, B2_total, M_total = run_shuffle(shuffles, repeats, k_fold, filepath, seperator)

# Base performances

Your dataset's base performances have been calculated. You can inspect the results in the upcomming cells. Please write the base performances down if you wish to later come back to the calculation.

All further cells can be independently calculated by providing preexisting base perfromance measures. The script enables you to calculate your specific B-Score (when providing your model's T-RMSE) and to calculate hypothetical B-Scores for your dataset using landmark T-RMSE values.

In [ ]:
#@title Inspect base performances (please note results!)

print('Your B1-RMSE is: ' + str(B1_total))
print('Your B2-RMSE is: ' + str(B2_total))
print('Your M-RMSE is: ' + str(M_total))

Please provide your pre-existing base performances below only if you DID NOT calculate them in this instance. In this case DO NOT change the values from None.

In [ ]:
new_B1_total = None
new_B2_total = None
new_M_total = None

In [ ]:
#@title Updating base performances

if new_B1_total != None:
  B1_total = new_B1_total
  B2_total = new_B2_total
  M_total = new_M_total

Please provide your model's T-RMSE score if you wish to do so. If not, please set the your_T-RMSE variable to None.

In [ ]:
your_T_RMSE = None

In [ ]:
#@title Inspecting the base performances

fig, ax1 = plt.subplots(1, 1, figsize = (8, 8), dpi = 100)
width = 0.7

if your_T_RMSE == None:
  labels = ['B1 RMSE', 'B2 RMSE', 'M RMSE']
  values = [B1_total, B2_total, M_total]
else:
  labels = ['B1 RMSE', 'B2 RMSE', 'M RMSE', 'T RMSE']
  values = [B1_total, B2_total, M_total, your_T_RMSE]

x = np.arange(len(values))
ax1.bar(x, values, width, color = ['royalblue','cornflowerblue','limegreen', 'tomato'])
ax1.set_ylabel('RMSE', fontsize = 14)
ax1.set_xticks(x)
ax1.set_xticklabels(labels, fontsize = 12) 
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.tick_params(axis = 'y', labelsize = 14)
ax1.set_title('Base performances', fontsize = 18)

In [ ]:
#@title Calculate hypothetical B-Scores for landmark T RMSE values

T_list = []
B_list = []
max = round(M_total * 10000)
for i in range(40000, max, 1):
  i = i / 10000
  B = calculate_B_Scores(B1_total, B2_total, M_total, i)
  T_list.append(i)
  B_list.append(B)
df = pd.DataFrame(np.column_stack([T_list, B_list]), columns = ['T', 'B'])

fig, ax1 = plt.subplots(1, 1, figsize = (8, 5.5), dpi = 100)
ax1.plot(df['T'], df['B'], c = 'black', lw = 4, zorder = -1)  
ax1.set_ylabel('B-Score', fontsize = 18)
ax1.set_xlabel('Hypothetical T-RMSE values', fontsize = 16)
ax1.tick_params(axis = 'both', labelsize = 16)
ax1.set_title('B-Scores from hypothetical T-RMSE values', fontsize = 18)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

T_list = []
B_list = []
for i in range(1, math.ceil(M_total - 1), 1):
  B = calculate_B_Scores(B1_total, B2_total, M_total, i)
  T_list.append(i)
  B_list.append(B)
df = pd.DataFrame(np.column_stack([T_list, B_list]), columns = ['T_RMSE', 'B_Score'])
df.head(50)




In [ ]:
#@title Calculate your B-Score

if your_T_RMSE == None:
  print('No RMSE provided!')

else:
  T_list = []
  B_list = []
  max = round(M_total * 10000)
  for i in range(40000, max, 1):
    i = i / 10000
    B = calculate_B_Scores(B1_total, B2_total, M_total, i)
    T_list.append(i)
    B_list.append(B)
  df = pd.DataFrame(np.column_stack([T_list, B_list]), columns = ['T', 'B'])

  fig, ax1 = plt.subplots(1, 1, figsize = (8, 5.5), dpi = 100)
  ax1.plot(df['T'], df['B'], c = 'black', lw = 4, zorder = -1)  
  ax1.set_ylabel('B-Score', fontsize = 18)
  ax1.set_xlabel('Hypothetical T-RMSE values', fontsize = 16)
  ax1.tick_params(axis = 'both', labelsize = 16)
  ax1.set_title('B-Scores from hypothetical T-RMSE values', fontsize = 18)
  ax1.spines['top'].set_visible(False)
  ax1.spines['right'].set_visible(False)
  B_Score = calculate_B_Scores(B1_total, B2_total, M_total, your_T_RMSE)
  ax1.scatter(your_T_RMSE, B_Score, c = 'red', lw = 11, zorder = 1)
  ax1.set_title('Your B-Score is: ' + str(round(B_Score, 3)), fontsize = 18)
  





# Thank you!

Thank you for using the complementary B-Score calculation script. We are looking forward to see your publication and your B-Score results.

To re-use the script (e.g. for DBP values) reload the complete script and go through it step by step from the beginning.